In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

In [2]:
# Use data from Random Forests lesson for Lending Club for 2015
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1,
    low_memory=False
)

In [3]:
# Remove two summary rows at the end that don't actual'id', 'member_idly contain data.
y2015 = y2015[:-2]

# Drop columns with nulls (lots of data to work with)
y2015 = y2015.dropna(axis=1)

In [4]:
# Convert ID and Interest Rate to numeric.
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

In [5]:
y2015.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'purpose', 'zip_code', 'addr_state', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
  

In [6]:
# Drop other columns with many unique variables
y2015.drop(['zip_code', 'earliest_cr_line'], 1, inplace=True)

In [7]:
y2015.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,4.210950e+05,421095.000000,421095.000000,421095.000000,421095.000000,...,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,4.210950e+05,4.210950e+05,421095.000000,4.210950e+05
mean,15240.285862,15240.285862,15234.156426,12.599315,441.842344,7.696561e+04,0.347397,0.572690,11.948643,0.233776,...,0.004042,0.094318,2.170947,93.924313,0.133188,0.064556,1.734070e+05,5.222199e+04,21958.208958,4.347865e+04
std,8571.325901,8571.325901,8567.935757,4.318782,244.847058,7.394996e+04,0.927695,0.867776,5.633119,0.649778,...,0.067965,0.522758,1.853858,8.875399,0.385975,0.444231,1.756722e+05,4.907494e+04,21840.371112,4.394271e+04
min,1000.000000,1000.000000,900.000000,5.320000,14.010000,0.000000e+00,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.500000e+03,0.000000e+00,0.000000,0.000000e+00
25%,8500.000000,8500.000000,8500.000000,9.170000,263.930000,4.600000e+04,0.000000,0.000000,8.000000,0.000000,...,0.000000,0.000000,1.000000,90.900000,0.000000,0.000000,5.090900e+04,2.259850e+04,7800.000000,1.548150e+04
50%,14000.000000,14000.000000,14000.000000,12.290000,385.410000,6.500000e+04,0.000000,0.000000,11.000000,0.000000,...,0.000000,0.000000,2.000000,97.400000,0.000000,0.000000,1.114050e+05,3.959600e+04,15200.000000,3.297000e+04
75%,20000.000000,20000.000000,20000.000000,15.590000,578.790000,9.169050e+04,0.000000,1.000000,15.000000,0.000000,...,0.000000,0.000000,3.000000,100.000000,0.000000,0.000000,2.496290e+05,6.565100e+04,28600.000000,5.829900e+04
max,35000.000000,35000.000000,35000.000000,28.990000,1445.460000,9.500000e+06,39.000000,6.000000,90.000000,86.000000,...,4.000000,39.000000,30.000000,100.000000,11.000000,85.000000,9.999999e+06,2.921551e+06,834300.000000,2.101913e+06


In [10]:
# blind approach
rfc = ensemble.RandomForestClassifier()
X = y2015.drop(['loan_status'], 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)


In [11]:
cross_val_score(rfc, X, Y, cv=5)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\

array([0.98169124, 0.9897769 , 0.98886237, 0.98955093, 0.98958618])

In [12]:
# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X, Y)

print(mlp.score(X, Y))
print(Y.value_counts()/len(Y))

cross_val_score(mlp, X, Y, cv=5)

0.9922226575950795
Fully Paid            0.711816
Charged Off           0.180014
Current               0.102825
Late (31-120 days)    0.003227
In Grace Period       0.001453
Late (16-30 days)     0.000663
Default               0.000002
Name: loan_status, dtype: float64


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


array([0.98942082, 0.99003811, 0.99268574, 0.99091643, 0.99223416])

In [13]:
# Try with multiple layers
mlp = MLPClassifier(hidden_layer_sizes=(500,50,100,))
mlp.fit(X, Y)

print(mlp.score(X, Y))
print(Y.value_counts()/len(Y))

cross_val_score(mlp, X, Y, cv=5)

0.71181562355288
Fully Paid            0.711816
Charged Off           0.180014
Current               0.102825
Late (31-120 days)    0.003227
In Grace Period       0.001453
Late (16-30 days)     0.000663
Default               0.000002
Name: loan_status, dtype: float64


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


array([0.71179739, 0.71180584, 0.71181087, 0.71181933, 0.71185656])

In [ ]:
# once again the multiple layers with fewer data points produces a much lower score
# the scores for random forest and for one layer network with many points are very similar
# and are high enough to suspect overfitting in both cases
# For this data set, random forest takes less time to run, so would pursue adjustments
# to that model to reduce risk of overfitting